# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

Este notebook foi projetado para guiar os alunos na realização das práticas de Big Data utilizando PySpark. Certifique-se de seguir cada etapa cuidadosamente para garantir a correta execução das atividades.

Seu trabalho começará na célula 5. Execute as 4 primeiras células para iniciar a atividade.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

Aqui, você fará o download do dataset necessário para as atividades. Certifique-se de que o arquivo foi descompactado corretamente antes de prosseguir.

In [1]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2025-03-31 11:28:41--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M  53.7MB/s    in 0.9s    

2025-03-31 11:28:47 (53.7 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
  inflating: imdb-reviews-pt-br.csv  


## Instalação manual das dependências para uso do pyspark no Google Colab

Esta etapa garante que todas as bibliotecas necessárias para o PySpark sejam instaladas no Google Colab.

In [2]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

A SparkSession é o ponto de entrada para usar o PySpark. Certifique-se de configurar corretamente o nome do aplicativo e o master.

In [83]:
from pyspark.sql import SparkSession

RU = "4568725"
appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

Não altere este código e use o dataframe imdb_df criado aqui em todo o seu trabalho. A criação de um dataframe diferente deste poderá causar erros na coluna sentiment e isso refletirá em erros de resposta das questões.

In [29]:
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
  StructField("id", StringType(), True),
  StructField("text_en", StringType(), True),
  StructField("text_pt", StringType(), True),
  StructField("sentiment", StringType(), True),
])


imdb_df: DataFrame = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

Nesta questão, você irá calcular a soma dos IDs para entradas onde o sentimento ('sentiment') é 'neg'.

### Objetivo:
- Usar a coluna 'sentiment' como chave e somar os valores da coluna 'id'.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave e o "id" como valor do tipo inteiro

A função map irá transformar cada linha do dataframe em uma **tupla** (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: coluna 'id' convertida para inteiro.

In [91]:
from pyspark.sql import DataFrame

def filter_negative_reviews(data: DataFrame) -> DataFrame:

    # (RU)
    RU = "4568725"

    # Exibe o RU no console
    print(f"RU: {RU}")

    # Filtra as linhas onde a coluna 'sentiment' é igual a 'neg' (avaliações negativas)
    return data.filter(data["sentiment"] == "neg")




## Cria funções de REDUCE:

- Criar função de reduce para somar os IDs por "sentiment".

A função reduce irá somar os valores dos IDs agrupados por chave ('sentiment').

In [92]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, sum

def sum_negative_ids(reviews: DataFrame) -> DataFrame:

    # Converte a coluna 'id' para o tipo inteiro, caso não esteja nesse formato
    reviews = reviews.withColumn("id", col("id").cast("int"))

    # Calcula a soma da coluna 'id' e retorna o resultado como DataFrame
    return reviews.select(sum("id"))


## Aplicação do map/reduce e visualização do resultado

Aqui, você aplicará as funções de map e reduce ao dataframe Spark para calcular os resultados. Não se esqueça de usar o método `.collect()` para visualizar os resultados.

In [93]:
# Filtra apenas as avaliações negativas do DataFrame imdb_df
negative_reviews = filter_negative_reviews(imdb_df)

# Calcula a soma dos IDs das avaliações negativas e coleta o valor resultante
sum_of_negative_ids = sum_negative_ids(negative_reviews).collect()[0][0]

# Exibe o resultado da soma dos IDs no console
print(f"Soma de IDs com reviews negativas: {sum_of_negative_ids}")


RU: 4568725
Soma de IDs com reviews negativas: 459568555


# Questão 2:

Nesta questão, você irá calcular a diferença no número total de palavras entre textos negativos em português e inglês.

### Objetivo:
- Contar as palavras em cada idioma (colunas 'text_pt' e 'text_en') para entradas onde o sentimento ('sentiment') é 'neg'.
- Subtrair o total de palavras em inglês do total em português.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave de uma tupla principal e como valor uma outra tupla com a soma das palavras de cada idioma como valor.

A função map irá transformar cada linha do dataframe em uma tupla (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: Nova tupla com:
  - Elemento 0: soma das palavras da coluna 'text_en'
  - Elemento 1: soma das palavras da coluna 'text_pt'

OU
- Chave: coluna 'sentiment'
- Valor: (soma das palavras da coluna 'text_pt') - (soma das palavras da coluna 'text_en')
  

Para contar as palavras deve-se primeiro separar os textos em uma lista de palavras para então descobrir o tamanho desta lista.
Dicas:

1. Use o método .split() e não .split(" ") de string para separar as palavras em uma lista ou use a função split(coluna de texto, regex) do pyspark com o regex igual à "[ ]+" ou "\s+"
2. Use len() para descobrir o tamanho da lista de palavras.

In [94]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, split, size

def map_sentiment_to_word_count(data: DataFrame) -> DataFrame:

    # Número de Registro do Usuário (RU)
    RU = "4568725"

    # Exibe o RU no console
    print(f"RU: {RU}")

    # Retorna um novo DataFrame com:
    # - A coluna 'sentiment'
    # - A contagem de palavras na coluna 'text_en'
    # - A contagem de palavras na coluna 'text_pt'
    return data.select(
        col("sentiment"),
        size(split(col("text_en"), "\\s+")).alias("text_en_word_count"),  # Conta palavras no texto em inglês
        size(split(col("text_pt"), "\\s+")).alias("text_pt_word_count")   # Conta palavras no texto em português
    )


## Cria funções de REDUCE:

- Criar função de reduce para somar o numero de palavras de cada texto português e inglês por "sentiment" (dependerá de como você optou por fazer sua função map2).

A função reduce irá somar os valores das quantidades de palavras agrupados por chave ('sentiment').

In [96]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import sum

def reduce_word_count_by_sentiment(sentiment_word_counts: DataFrame) -> DataFrame:
    # Agrupa os dados pelo campo 'sentiment' e soma as contagens de palavras
    return sentiment_word_counts.groupBy("sentiment").agg(
        # Soma a contagem de palavras dos textos em inglês e renomeia a coluna
        sum("text_en_word_count").alias("total_text_en_words"),

        # Soma a contagem de palavras dos textos em português e renomeia a coluna
        sum("text_pt_word_count").alias("total_text_pt_words")
    )


## Aplicação do map/reduce e visualização do resultado

1. Aplicar o map/reduce no seu dataframe spark e realizar o collect() ao final
2. Selecionar os dados referentes aos textos negativos para realizar a subtração.
3. Realizar a subtração das contagens de palavras dos textos negativos para obter o resultado final

In [97]:
# Filtra apenas as avaliações negativas do DataFrame imdb_df
negative_data = filter_negative_reviews(imdb_df)

# Mapeia os sentimentos para a contagem de palavras nos textos em inglês e português
sentiment_word_counts = map_sentiment_to_word_count(negative_data)

# Reduz os dados somando o total de palavras nos textos em inglês e português, agrupados por sentiment
total_word_counts = reduce_word_count_by_sentiment(sentiment_word_counts)

# Coleta o primeiro resultado do DataFrame como uma linha de dados
result = total_word_counts.collect()[0]

# Calcula a diferença entre a contagem de palavras nos textos em português e em inglês
word_count_difference = result[2] - result[1]

# Exibe a diferença calculada no console
print(f"Diferença entre Texto em Inglês e em Português:")
print(f"                    {word_count_difference}")



RU: 4568725
RU: 4568725
Diferença entre Texto em Inglês e em Português:
                    54976
